# Model Server

## Environment

In [1]:
import nuclio

In [2]:
import os

base_path = os.path.abspath('../')
data_path = os.path.join(base_path, 'data')
src_path = os.path.join(base_path, 'src')
streaming_path = os.path.join(base_path, 'streaming')
os.environ['base_path'] = base_path
os.environ['data_path'] = data_path
os.environ['src_path'] = src_path
os.environ['streaming_path'] = streaming_path
os.environ['fs_streaming_path'] = os.path.join(base_path, 'streaming')

In [3]:
%nuclio config kind = "nuclio"
%nuclio config spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [4]:
%%nuclio env

# streaming/features
METRICS_TABLE = ${data_path}
FEATURES_TABLE = ${streaming_path}/features
PREDICTIONS_TABLE = ${streaming_path}/predictions

# Define number of batches to keep the demo running for (-1 will run forever)
BATCHES_TO_GENERATE = 20

# Model
model_path = ${base_path}/artifacts/model/1/model.pkl
model_name = netops_predictor_v1
model_class = MLRunModel
model_col = model
model_class_col = class
prediction_col = predictions
worker_col = worker
hostname_col = hostname
timestamp_col = when
orig_timestamp_col = timestamp

%nuclio: setting 'METRICS_TABLE' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting 'PREDICTIONS_TABLE' environment variable
%nuclio: setting 'model_path' environment variable
%nuclio: setting 'model_name' environment variable
%nuclio: setting 'model_class' environment variable
%nuclio: setting 'model_col' environment variable
%nuclio: setting 'model_class_col' environment variable
%nuclio: setting 'prediction_col' environment variable
%nuclio: setting 'worker_col' environment variable
%nuclio: setting 'hostname_col' environment variable
%nuclio: setting 'timestamp_col' environment variable
%nuclio: setting 'orig_timestamp_col' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

In [5]:
# nuclio: start-code

In [6]:
import os
import pandas as pd
import cloudpickle
import numpy as np
import json
from mlrun import get_or_create_ctx
import socket

In [7]:
def get_data_parquet(context):
    mpath = [os.path.join(context.features_table, file) for file in os.listdir(context.features_table) if file.endswith(('parquet', 'pq'))]
    files_by_updated = sorted(mpath, key=os.path.getmtime, reverse=True)
    return pd.read_parquet(files_by_updated[:1][0])

In [8]:
def save_to_parquet(context, df: pd.DataFrame):
    print('Saving features to Parquet')
    
    # Need to fix timestamps from ns to ms if we write to parquet 
    # And add this model name to indexes
    keys = list([name if name != context.orig_timestamp_col else context.timestamp_col for name in df.index.names]) + [context.model_col, context.model_class_col, context.worker_col, context.hostname_col]
    df = df.reset_index()
    df[context.timestamp_col] = df.pop(context.orig_timestamp_col).astype('datetime64[ms]')
    
    # Fix indexes
    df = df.set_index(keys)
    
    # Save parquet
    first_timestamp = df.index[0][0].strftime('%Y%m%dT%H%M%S')
    last_timestamp = df.index[-1][0].strftime('%Y%m%dT%H%M%S')
    filename = first_timestamp + '-' + last_timestamp + '.parquet'
    filepath = os.path.join(context.predictions_table, filename)
    with open(filepath, 'wb+') as f:
        df.to_parquet(f)

In [9]:
def init_context(context):
    
    # How many batches to create? (-1 will run forever)
    batches_to_generate = int(os.getenv('BATCHES_TO_GENERATE', 20))
    setattr(context, 'batches_to_generate', batches_to_generate)
    setattr(context, 'batches_generated', 0)
    
    # Set vars from env
    setattr(context, 'model_name', os.getenv('model_name', 'netops_model'))
    setattr(context, 'model_col', os.getenv('model_col', 'model'))
    setattr(context, 'model_class_col', os.getenv('model_class_col', 'class'))
    setattr(context, 'worker_col', os.getenv('worker_col', 'worker'))
    setattr(context, 'hostname_col', os.getenv('hostname_col', 'hostname'))
    setattr(context, 'timestamp_col', os.getenv('timestamp_col', 'when'))
    setattr(context, 'orig_timestamp_col', os.getenv('orig_timestamp_col', 'timestamp'))
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    setattr(context, 'predictions_table', os.getenv('PREDICTIONS_TABLE', 'netops_predictions'))
    setattr(context, 'prediction_col', os.getenv('prediction_col', 'prediction'))
    
    # Load model
    model_path = os.environ['model_path']
    if model_path.startswith('store://'):
        mlctx = get_or_create_ctx('inference')
        model = mlctx.get_dataitem(model_path)
        model_path = os.path.join(model.url, 'model.pkl')
    with open(model_path, 'rb') as f:
        model = cloudpickle.load(f)
    setattr(context, 'model', model)
    setattr(context, 'model_class', type(model).__name__)
    
     # Create saving directory if needed
    filepath = os.path.join(context.predictions_table)
    if not os.path.exists(filepath):
        os.makedirs(filepath)

In [10]:
def handler(context, event):
    
    # Limit the number of generated batches to save cluster resources
    # for people forgetting the demo running
    if (context.batches_to_generate == -1) or (context.batches_generated <= context.batches_to_generate):
    
        if getattr(event.trigger, 'kind', 'cron') == 'cron':
            # Get latest parquets
            df = get_data_parquet(context)

            # Predict
            df[context.prediction_col] = context.model.predict(df.values)

            # Add server metadata
            df[context.model_col] = context.model_name
            df[context.model_class_col] = context.model_class
            df[context.worker_col] = context.worker_id
            df[context.hostname_col] = socket.gethostname()

            # Save
            save_to_parquet(context, df)
        else:
            body = json.loads(event.body)
            feats = np.asarray(body['instances'])
            result: np.ndarray = context.model.predict(feats)
            return result.tolist()
        
        # Update batches count
        context.batches_generated += 1

In [64]:
# nuclio: end-code

## Local test

In [69]:
init_context(context)

In [72]:
event = nuclio.Event(body='', trigger={'kind': 'cron'})
out = handler(context, event)
out

Saving features to Parquet


## Test

In [11]:
from mlrun import code_to_function, mount_v3io

In [12]:
fn = code_to_function('inference-server',
                      kind='nuclio',
                      project='network-operations')
fn.spec.base_spec['spec']['build']['baseImage'] = 'mlrun/ml-models'
fn.apply(mount_v3io())
fn.add_trigger('cron', nuclio.triggers.CronTrigger(interval='1m'))

In [13]:
fn.save()
fn.export('../src/inference-server.yaml')

> 2020-12-22 10:18:10,117 [info] function spec saved to path: ../src/inference-server.yaml


In [14]:
fn.deploy(project='network-operations')

> 2020-12-22 10:18:10,122 [info] Starting remote function deploy
2020-12-22 10:18:10  (info) Deploying function
2020-12-22 10:18:10  (info) Building
2020-12-22 10:18:10  (info) Staging files and preparing base images
2020-12-22 10:18:10  (info) Building processor image
2020-12-22 10:18:11  (info) Build complete
2020-12-22 10:18:19  (info) Function deploy complete
> 2020-12-22 10:18:19,657 [info] function deployed, address=default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:30156


'http://default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:30156'

In [25]:
pd.read_parquet('../streaming/predictions/20200630T064217-20200630T074212.parquet')

cpu_utilization  \
when                    company                     data_center      device        model               class                  worker hostname                                    
2020-06-30 06:42:17.454 Wilson_LLC                  Zachary_Drives   6001003522699 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        66.939139   
                                                    Obrien_Mountain  0966571261270 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        72.492707   
                                                                     8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.116878   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        64.944107   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        78.641128   
...                                                                                                                                                                        ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Obrien_Mountain  8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        59.574487   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd       100.000000   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd       100.000000   
                                                    Dominique_Branch 4579248894449 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.053014   
                                                                     7731097392608 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.629988   

                                                                                                                                                                  latency  \
when                    company                     data_center      device        model               class                  worker hostname                               
2020-06-30 06:42:17.454 Wilson_LLC                  Zachary_Drives   6001003522699 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.537279   
                                                    Obrien_Mountain  0966571261270 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
                                                                     8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    2.606934   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    1.571046   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
...                                                                                                                                                                   ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Obrien_Mountain  8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd  100.000000   
                                                                     4285071567351 netops_predictor_v1 RandomFores